# GOES-16 Data

- Reading NetCDF files with a remapping function (`remap`)
- Plotting data with `Basemap`
  
Based on [GEONETClass: Manipulating GOES-16 Data Wth Python (GNC-A)](https://geonetcast.wordpress.com/2017/04/27/geonetclass-manipulating-goes-16-data-with-python-part-i/).

## Loading necessary packages

In [1]:
import matplotlib.pyplot as plt
from netCDF4 import Dataset
from mpl_toolkits.basemap import Basemap
import numpy as np
from datetime import datetime

from remap import remap

from cpt_convert import loadCPT
from matplotlib.colors import LinearSegmentedColormap

from extracting_band_info import extract_band_info

## Defining necessary functions

### `get_info_file`

Using the name of the NetCDF file (*filename*), extract:

- According to the band:
    - *band_unit*, unit of the variable to be shown on the plot
    - *band_conversion*, value to be added to convert from Kelvin to Celsius, if necessary
    - *band_cpt*, color palette of the variable for the plot
    - *band_minvalue*, minimum value of the variable for the plot
    - *band_maxvalue*, maximum value of the variable for the plot
- *title*, string to be used on the plot title
- *name*, string to define the name of the saved figure

In [2]:
def get_info_file(filename, fig_type):
    print('Getting info from ' + filename)
    
    band = (filename[filename.find('M3C') + 3:filename.find('_G16')])
    band_cw, band_unit, band_conversion, band_cpt, band_minvalue, band_maxvalue = extract_band_info(band)
    
    start_timestamp = (filename[filename.find('_s') + 2:filename.find('_e')])
    start_timestamp = datetime.strptime(start_timestamp[:13], '%Y%j%H%M%S')
    end_timestamp = (filename[filename.find('_e') + 2:filename.find('_c')])
    end_timestamp = datetime.strptime(end_timestamp[:13], '%Y%j%H%M%S')
    
    title = 'GOES-16 ABI Band ' + band + ' ' + band_cw + '\n Scan from ' + str(start_timestamp) + ' to ' + str(end_timestamp.time()) + ' UTC'
    name = 'Band_' + band + '/GOES16_B'+ band + '_' + fig_type + '_SD' + datetime.strftime(start_timestamp, '%Y%m%d%H%M') + '.png'
    
    return band_unit, band_conversion, band_cpt, band_minvalue, band_maxvalue, title, name

### `read_define_bounds_netcdf`

Using the NetCDF file:

- Read with `ncdf4` to extract information about the data extent
- Regrid to rectangular projection using `remap`

**ATTENTION: If the files are from the Operational Mode (starting December 2017), `remap.py` should be altered! (line 15)**

In [3]:
def read_define_bounds_netcdf(file, band_conversion, extent):
    print('Reading NetCDF file ' + file)
    
    nc = Dataset(file)
 
    #-- Visualization extent for Full Disk
    # geo_extent = nc.variables['geospatial_lat_lon_extent']
    # min_lon = float(geo_extent.geospatial_westbound_longitude)
    # max_lon = float(geo_extent.geospatial_eastbound_longitude)
    # min_lat = float(geo_extent.geospatial_southbound_latitude)
    # max_lat = float(geo_extent.geospatial_northbound_latitude)
    # extent = [min_lon, min_lat, max_lon, max_lat]
    
    resolution = 2
 
    #-- Image extent required for the reprojection
    H = nc.variables['goes_imager_projection'].perspective_point_height
    x1 = nc.variables['x_image_bounds'][0] * H # x1 = -5434894.885056 
    x2 = nc.variables['x_image_bounds'][1] * H # x2 = 5434894.885056
    y1 = nc.variables['y_image_bounds'][1] * H # y1 = -5434894.885056
    y2 = nc.variables['y_image_bounds'][0] * H # y2 = 5434894.885056
    grid = remap(file, extent, resolution, x1, y1, x2, y2) 
    
    data = grid.ReadAsArray() + band_conversion
    
    return data, extent

### `plot_save_figure`

Using data and derived variables:

- Plot using shapefiles *ne_10m_admin_0_countries* (global) and *estadosl_2007* (Brazilian states), as well as pre-defined spacing according to image boundaries (*grid_spacing*)
- Save the resulting figure with transparent background

In [4]:
def plot_save_figure(data, extent, shapefile, grid_spacing, cpt, min_value, max_value, title, band_unit, name):
    fig = plt.figure(figsize=(5, 6))
    fig.set_facecolor("w")
    ax = fig.add_subplot(111)

    bmap = Basemap(llcrnrlon=extent[0], llcrnrlat=extent[1], urcrnrlon=extent[2], urcrnrlat=extent[3], epsg=4326)

    bmap.readshapefile(shapefile + 'ne_10m_admin_0_countries', 'ne_10m_admin_0_countries', linewidth=0.5, color='darkslategray')
    bmap.readshapefile(shapefile + 'estadosl_2007', 'estadosl_2007', linewidth=0.3, color='darkslategray')
    bmap.drawparallels(np.arange(-90.0, 90.0, grid_spacing), linewidth=0.25, color='white', labels=[True,False,False,True])
    bmap.drawmeridians(np.arange(0.0, 360.0, grid_spacing), linewidth=0.25, color='white', labels=[True,False,False,True])

    cpt_convert = LinearSegmentedColormap('cpt', cpt)

    bmap.imshow(data, origin='upper', cmap=cpt_convert, vmin=min_value, vmax=max_value)

    plt.title(title.decode("utf8"), weight='bold', stretch='condensed', size='large')
    bmap.colorbar(location='right', label=band_unit.decode("utf8"))
    
    print('Saving figure in ' + name)
    plt.savefig(name, dpi=300, transparent=True, bbox_inches='tight')
    plt.close()
    
    print('--------------------------------------------------------------------------------------------------------------')

## Defining filepaths and custom variables

In [5]:
shapefile_path = '../Data/GENERAL/shapefiles/'
files_path = '../Data/SATELLITE/GOES16/level_2/2017/'
save_path = 'figures/'

filenames = open('g16_filenames.txt').read().split('\n')[:-1]

#-- Custom visualization extent and grid spacing
#-- South America
extent = [-85., -60., -30., 15.] #-- [min lon, min lat, max lon, max lat]
grid_spacing = 10.0
fig_type = 'SA'

## Processing all the files in *filenames*

In [ ]:
for filename in filenames:
    unit, conversion, cpt, minvalue, maxvalue, fig_title, fig_name = get_info_file(filename, fig_type)
    
    sat_data, extent = read_define_bounds_netcdf(files_path + filename, conversion, extent)
    
    plot_save_figure(sat_data, extent, shapefile_path, grid_spacing, cpt, minvalue, maxvalue, fig_title, unit, save_path + fig_name) 

Getting info from OR_ABI-L2-CMIPF-M3C01_G16_s20170710000027_e20170710010394_c20170710010461.nc
Reading NetCDF file ../Data/SATELLITE/GOES16/level_2/2017/OR_ABI-L2-CMIPF-M3C01_G16_s20170710000027_e20170710010394_c20170710010461.nc
Remapping ../Data/SATELLITE/GOES16/level_2/2017/OR_ABI-L2-CMIPF-M3C01_G16_s20170710000027_e20170710010394_c20170710010461.nc
- finished! Time: 0.922801017761 seconds
Saving figure in figures/Band_01/GOES16_B01_SA_SD201703120000.png
--------------------------------------------------------------------------------------------------------------
Getting info from OR_ABI-L2-CMIPF-M3C01_G16_s20170710015027_e20170710025394_c20170710025466.nc
Reading NetCDF file ../Data/SATELLITE/GOES16/level_2/2017/OR_ABI-L2-CMIPF-M3C01_G16_s20170710015027_e20170710025394_c20170710025466.nc
Remapping ../Data/SATELLITE/GOES16/level_2/2017/OR_ABI-L2-CMIPF-M3C01_G16_s20170710015027_e20170710025394_c20170710025466.nc
- finished! Time: 0.968336105347 seconds
Saving figure in figures/Band_0